# 05_Dimensionality_Reduction.py

This notebook was automatically converted from a Python script.

In [ ]:

"""
Scikit-learn Boyut İndirgeme Teknikleri

Bu script, scikit-learn kütüphanesindeki boyut indirgeme tekniklerini göstermektedir.
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_digits, load_iris, fetch_olivetti_faces
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score



Boyut İndirgeme Teknikleri


In [ ]:
from sklearn.decomposition import PCA, KernelPCA, NMF, TruncatedSVD
from sklearn.manifold import TSNE, MDS, Isomap, LocallyLinearEmbedding
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis



%matplotlib inline


In [ ]:

print("Scikit-learn Boyut İndirgeme Teknikleri")
print("="*50)



1. Veri Yükleme ve Hazırlama


In [ ]:
print("\n1. Veri Yükleme ve Hazırlama")
print("-"*50)



Rakam veri seti


In [ ]:
digits = load_digits()
X_digits = digits.data
y_digits = digits.target



Iris veri seti


In [ ]:
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

print("Rakam veri seti boyutu:", X_digits.shape)
print("Iris veri seti boyutu:", X_iris.shape)



Rakam örneklerini görselleştirme


In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(digits.images[i], cmap='binary')
    ax.set_title(f'Digit: {digits.target[i]}')
    ax.axis('off')
    
plt.tight_layout()
plt.savefig('digit_examples.png')
plt.close()
print("Rakam örnekleri görselleştirmesi 'digit_examples.png' olarak kaydedildi.")



2. Temel Bileşen Analizi (PCA)


In [ ]:
print("\n2. Temel Bileşen Analizi (PCA)")
print("-"*50)



Iris veri seti üzerinde PCA


Veriyi ölçeklendirme


In [ ]:
scaler = StandardScaler()
X_iris_scaled = scaler.fit_transform(X_iris)



PCA uygulama


In [ ]:
pca = PCA()
X_iris_pca = pca.fit_transform(X_iris_scaled)



Açıklanan varyans oranı


In [ ]:
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

plt.figure(figsize=(12, 5))

plt.subplot(121)
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, alpha=0.7)
plt.step(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio, where='mid', color='red')
plt.xlabel('Temel Bileşen')
plt.ylabel('Açıklanan Varyans Oranı')
plt.title('Iris Veri Seti - Temel Bileşenlerin Açıklanan Varyans Oranı')
plt.grid(True)

plt.subplot(122)
plt.scatter(X_iris_pca[:, 0], X_iris_pca[:, 1], c=y_iris, cmap='viridis', s=50, alpha=0.8)
plt.xlabel(f'Birinci Temel Bileşen ({explained_variance_ratio[0]:.2%})')
plt.ylabel(f'İkinci Temel Bileşen ({explained_variance_ratio[1]:.2%})')
plt.title('Iris Veri Seti - İlk İki Temel Bileşen')
plt.colorbar(label='Sınıf')
plt.grid(True)

plt.tight_layout()
plt.savefig('pca_iris.png')
plt.close()
print("PCA analizi görselleştirmesi 'pca_iris.png' olarak kaydedildi.")



Yük (loadings) matrisini görselleştirme


In [ ]:
pca_loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f'PC{i+1}' for i in range(pca.n_components_)],
    index=iris.feature_names
)

plt.figure(figsize=(10, 6))
sns.heatmap(pca_loadings, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('PCA Yük Matrisi')
plt.tight_layout()
plt.savefig('pca_loadings.png')
plt.close()
print("PCA yük matrisi görselleştirmesi 'pca_loadings.png' olarak kaydedildi.")



3. Doğrusal Ayırt Edici Analiz (LDA)


In [ ]:
print("\n3. Doğrusal Ayırt Edici Analiz (LDA)")
print("-"*50)



LDA - Sınıf bilgisini kullanan denetimli boyut indirgeme


In [ ]:
lda = LinearDiscriminantAnalysis(n_components=2)
X_iris_lda = lda.fit_transform(X_iris_scaled, y_iris)



LDA sonuçlarını görselleştirme


In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(121)
plt.scatter(X_iris_pca[:, 0], X_iris_pca[:, 1], c=y_iris, cmap='viridis', s=50, alpha=0.8)
plt.xlabel('Birinci Temel Bileşen')
plt.ylabel('İkinci Temel Bileşen')
plt.title('Iris Veri Seti - PCA')
plt.grid(True)

plt.subplot(122)
plt.scatter(X_iris_lda[:, 0], X_iris_lda[:, 1], c=y_iris, cmap='viridis', s=50, alpha=0.8)
plt.xlabel('Birinci LDA Bileşeni')
plt.ylabel('İkinci LDA Bileşeni')
plt.title('Iris Veri Seti - LDA')
plt.grid(True)

plt.tight_layout()
plt.savefig('pca_vs_lda.png')
plt.close()
print("PCA ve LDA karşılaştırması görselleştirmesi 'pca_vs_lda.png' olarak kaydedildi.")



4. t-SNE (t-Distributed Stochastic Neighbor Embedding)


In [ ]:
print("\n4. t-SNE (t-Distributed Stochastic Neighbor Embedding)")
print("-"*50)



Rakam veri seti üzerinde t-SNE uygulama


Veriyi ölçeklendirme


In [ ]:
X_digits_scaled = StandardScaler().fit_transform(X_digits)



t-SNE uygulama (yavaş olabilir, özellikle büyük veri setlerinde)


In [ ]:
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_digits_tsne = tsne.fit_transform(X_digits_scaled)



t-SNE sonuçlarını görselleştirme


In [ ]:
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_digits_tsne[:, 0], X_digits_tsne[:, 1], c=y_digits, cmap='tab10', s=50, alpha=0.8)
plt.colorbar(scatter, label='Rakam')
plt.title('Rakam Veri Seti - t-SNE ile Görselleştirme')
plt.grid(True)
plt.tight_layout()
plt.savefig('tsne_digits.png')
plt.close()
print("t-SNE analizi görselleştirmesi 'tsne_digits.png' olarak kaydedildi.")



5. Farklı Perplexity Değerleri ile t-SNE


In [ ]:
print("\n5. Farklı Perplexity Değerleri ile t-SNE")
print("-"*50)



Farklı perplexity değerleri ile t-SNE


In [ ]:
perplexities = [5, 30, 50, 100]
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, perplexity in enumerate(perplexities):
    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity)
    X_digits_tsne = tsne.fit_transform(X_digits_scaled[:500])  # İlk 500 örnek (daha hızlı)
    
    scatter = axes[i].scatter(X_digits_tsne[:, 0], X_digits_tsne[:, 1], 
                              c=y_digits[:500], cmap='tab10', s=40, alpha=0.8)
    axes[i].set_title(f'Perplexity = {perplexity}')
    axes[i].grid(True)
    
fig.colorbar(scatter, ax=axes, label='Rakam')
plt.suptitle('t-SNE ile Farklı Perplexity Değerleri Karşılaştırması', fontsize=16)
plt.tight_layout()
plt.savefig('tsne_perplexity_comparison.png')
plt.close()
print("Farklı perplexity değerleri ile t-SNE karşılaştırması 'tsne_perplexity_comparison.png' olarak kaydedildi.")



6. Kernel PCA


In [ ]:
print("\n6. Kernel PCA")
print("-"*50)



Kernel PCA - doğrusal olmayan boyut indirgeme


In [ ]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, kernel in enumerate(kernels):
    kpca = KernelPCA(n_components=2, kernel=kernel, random_state=42)
    X_iris_kpca = kpca.fit_transform(X_iris_scaled)
    
    scatter = axes[i].scatter(X_iris_kpca[:, 0], X_iris_kpca[:, 1], 
                              c=y_iris, cmap='viridis', s=50, alpha=0.8)
    axes[i].set_title(f'Kernel PCA - {kernel} çekirdeği')
    axes[i].grid(True)
    
fig.colorbar(scatter, ax=axes, label='Sınıf')
plt.suptitle('Farklı Çekirdek Fonksiyonları ile Kernel PCA', fontsize=16)
plt.tight_layout()
plt.savefig('kernel_pca_comparison.png')
plt.close()
print("Farklı çekirdeklerle Kernel PCA karşılaştırması 'kernel_pca_comparison.png' olarak kaydedildi.")



7. MDS (Multidimensional Scaling)


In [ ]:
print("\n7. MDS (Multidimensional Scaling)")
print("-"*50)



MDS uygulama


In [ ]:
mds = MDS(n_components=2, random_state=42)
X_iris_mds = mds.fit_transform(X_iris_scaled)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_iris_mds[:, 0], X_iris_mds[:, 1], c=y_iris, cmap='viridis', s=50, alpha=0.8)
plt.colorbar(scatter, label='Sınıf')
plt.title('Iris Veri Seti - MDS ile Görselleştirme')
plt.grid(True)
plt.tight_layout()
plt.savefig('mds_iris.png')
plt.close()
print("MDS analizi görselleştirmesi 'mds_iris.png' olarak kaydedildi.")



8. Isomap


In [ ]:
print("\n8. Isomap")
print("-"*50)



Isomap uygulama


In [ ]:
isomap = Isomap(n_components=2, n_neighbors=10)
X_iris_isomap = isomap.fit_transform(X_iris_scaled)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_iris_isomap[:, 0], X_iris_isomap[:, 1], c=y_iris, cmap='viridis', s=50, alpha=0.8)
plt.colorbar(scatter, label='Sınıf')
plt.title('Iris Veri Seti - Isomap ile Görselleştirme')
plt.grid(True)
plt.tight_layout()
plt.savefig('isomap_iris.png')
plt.close()
print("Isomap analizi görselleştirmesi 'isomap_iris.png' olarak kaydedildi.")



9. Boyut İndirgeme Tekniklerinin Karşılaştırılması


In [ ]:
print("\n9. Boyut İndirgeme Tekniklerinin Karşılaştırılması")
print("-"*50)



Boyut indirgeme tekniklerinin karşılaştırılması


In [ ]:
techniques = [
    ('PCA', PCA(n_components=2)),
    ('LDA', LinearDiscriminantAnalysis(n_components=2)),
    ('Kernel PCA (RBF)', KernelPCA(n_components=2, kernel='rbf')),
    ('t-SNE', TSNE(n_components=2, random_state=42)),
    ('MDS', MDS(n_components=2, random_state=42)),
    ('Isomap', Isomap(n_components=2, n_neighbors=10))
]

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, (name, technique) in enumerate(techniques):
    if name == 'LDA':
        X_transformed = technique.fit_transform(X_iris_scaled, y_iris)
    else:
        X_transformed = technique.fit_transform(X_iris_scaled)
    
    scatter = axes[i].scatter(X_transformed[:, 0], X_transformed[:, 1], 
                              c=y_iris, cmap='viridis', s=50, alpha=0.8)
    axes[i].set_title(name)
    axes[i].grid(True)
    
fig.colorbar(scatter, ax=axes, label='Sınıf')
plt.suptitle('Iris Veri Seti - Boyut İndirgeme Teknikleri Karşılaştırması', fontsize=16)
plt.tight_layout()
plt.savefig('dimensionality_reduction_comparison.png')
plt.close()
print("Boyut indirgeme teknikleri karşılaştırması 'dimensionality_reduction_comparison.png' olarak kaydedildi.")



10. Sınıflandırma Performansı Karşılaştırması


In [ ]:
print("\n10. Sınıflandırma Performansı Karşılaştırması")
print("-"*50)



Boyut indirgeme tekniklerinin sınıflandırma performansı üzerindeki etkisi


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits, test_size=0.3, random_state=42)



Test edilecek bileşen sayıları


In [ ]:
components = [2, 5, 10, 20, 30, 40, 50, 64]
techniques = [
    ('PCA', PCA),
    ('Kernel PCA (RBF)', lambda n_components: KernelPCA(n_components=n_components, kernel='rbf')),
    ('Truncated SVD', TruncatedSVD)
]

results = {}

for name, technique in techniques:
    scores = []
    for n_components in components:
        if n_components > min(X_train.shape):
            scores.append(None)
            continue
            
        # Veri ölçeklendirme ve boyut indirgeme pipeline'ı
        dim_reduction = Pipeline([
            ('scaler', StandardScaler()),
            ('reduction', technique(n_components=n_components)),
            ('classifier', KNeighborsClassifier(n_neighbors=5))
        ])
        
        # Model eğitimi ve değerlendirme
        dim_reduction.fit(X_train, y_train)
        y_pred = dim_reduction.predict(X_test)
        scores.append(accuracy_score(y_test, y_pred))
    
    results[name] = scores



Sonuçları görselleştirme


In [ ]:
plt.figure(figsize=(12, 7))
for name, scores in results.items():
    valid_components = [c for i, c in enumerate(components) if scores[i] is not None]
    valid_scores = [s for s in scores if s is not None]
    plt.plot(valid_components, valid_scores, marker='o', linestyle='-', linewidth=2, label=name)

plt.xlabel('Bileşen Sayısı')
plt.ylabel('Doğruluk')
plt.title('Boyut İndirgeme Tekniklerinin Sınıflandırma Performansı Üzerindeki Etkisi')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('dimensionality_reduction_classification_performance.png')
plt.close()
print("Boyut indirgeme teknikleri sınıflandırma performansı karşılaştırması 'dimensionality_reduction_classification_performance.png' olarak kaydedildi.") 
